<a href="https://colab.research.google.com/github/lucacamus13/dashboard-ARG/blob/main/inflaci%C3%B3n_ARG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install streamlit


# Inflación

In [37]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime

def obtener_datos_indec_completo():
    """
    Lee las 3 hojas principales del Excel del INDEC:
    - Hoja 0: Variación mensual
    - Hoja 1: Variación interanual
    - Hoja 2: Índices IPC
    """
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_07_25.xls"

    try:
        print(f"Descargando Excel desde: {url}")

        response = requests.get(url, timeout=30)
        response.raise_for_status()

        print(f"✓ Archivo descargado: {len(response.content)} bytes")

        # Leer las 3 hojas principales
        hojas_datos = {}
        nombres_hojas = [
            'Variación mensual IPC Nacional',
            'Var. interanual IPC Nacional',
            'Índices IPC Cobertura Nacional'
        ]

        for i, nombre in enumerate(nombres_hojas):
            try:
                df = pd.read_excel(BytesIO(response.content), sheet_name=i, engine='xlrd')
                hojas_datos[nombre] = df
                print(f"✓ Hoja {i} '{nombre}': {len(df)} filas x {len(df.columns)} columnas")
            except Exception as e:
                print(f"❌ Error leyendo hoja {i}: {e}")

        return hojas_datos

    except Exception as e:
        print(f"❌ Error: {e}")
        return {}

def extraer_nivel_general_de_hoja(df, nombre_hoja):
    """
    Extrae datos de 'Nivel general' de cualquier hoja
    """
    if df.empty:
        return pd.DataFrame()

    print(f"\n=== PROCESANDO {nombre_hoja.upper()} ===")

    # Buscar fila con fechas
    fila_fechas = None
    for i, fila in df.iterrows():
        fechas_count = sum(1 for val in fila if isinstance(val, datetime))
        if fechas_count > 10:
            fila_fechas = i
            break

    # Buscar fila "Nivel general"
    fila_nivel_general = None
    for i, fila in df.iterrows():
        if pd.notna(fila.iloc[0]) and str(fila.iloc[0]).strip() == 'Nivel general':
            fila_nivel_general = i
            break

    if fila_fechas is None or fila_nivel_general is None:
        print(f"❌ No se encontraron datos en {nombre_hoja}")
        return pd.DataFrame()

    print(f"✓ Fechas en fila {fila_fechas}, Nivel general en fila {fila_nivel_general}")

    # Extraer fechas y valores
    fechas = df.iloc[fila_fechas].values[1:]  # Saltar primera columna
    valores = df.iloc[fila_nivel_general].values[1:]  # Saltar primera columna

    # Crear DataFrame
    datos = []
    for fecha, valor in zip(fechas, valores):
        if pd.notna(fecha) and pd.notna(valor) and isinstance(fecha, datetime):
            datos.append({
                'fecha': fecha,
                'valor': float(valor)
            })

    if datos:
        df_resultado = pd.DataFrame(datos).sort_values('fecha').reset_index(drop=True)
        print(f"✓ {len(df_resultado)} registros extraídos")
        return df_resultado

    return pd.DataFrame()

def combinar_datos_inflacion(hojas_datos):
    """
    Combina datos de las 3 hojas en un DataFrame completo
    """
    print("\n=== COMBINANDO DATOS DE LAS 3 HOJAS ===")

    # Extraer datos de cada hoja
    datos_combinados = {}

    # Hoja 0: Variación mensual
    if 'Variación mensual IPC Nacional' in hojas_datos:
        df_mensual = extraer_nivel_general_de_hoja(
            hojas_datos['Variación mensual IPC Nacional'],
            'Variación mensual'
        )
        if not df_mensual.empty:
            datos_combinados['variacion_mensual'] = df_mensual.set_index('fecha')['valor']

    # Hoja 1: Variación interanual
    if 'Var. interanual IPC Nacional' in hojas_datos:
        df_interanual = extraer_nivel_general_de_hoja(
            hojas_datos['Var. interanual IPC Nacional'],
            'Variación interanual'
        )
        if not df_interanual.empty:
            datos_combinados['variacion_interanual'] = df_interanual.set_index('fecha')['valor']

    # Hoja 2: Índices IPC
    if 'Índices IPC Cobertura Nacional' in hojas_datos:
        df_indices = extraer_nivel_general_de_hoja(
            hojas_datos['Índices IPC Cobertura Nacional'],
            'Índices IPC'
        )
        if not df_indices.empty:
            datos_combinados['ipc_indice'] = df_indices.set_index('fecha')['valor']

    if not datos_combinados:
        print("❌ No se pudieron extraer datos de ninguna hoja")
        return pd.DataFrame()

    # Combinar todos los datos
    df_final = pd.DataFrame(datos_combinados)
    df_final = df_final.reset_index()

    # Rellenar valores faltantes si es necesario
    df_final = df_final.dropna(subset=['fecha'])

    print(f"✓ DataFrame final: {len(df_final)} registros")
    print(f"✓ Columnas: {list(df_final.columns)}")

    return df_final

def actualizar_inflacion_completa():
    """
    Función principal que obtiene datos completos de inflación
    """
    print("=== OBTENIENDO DATOS COMPLETOS DE INFLACIÓN DEL INDEC ===")

    # Descargar y leer todas las hojas
    hojas_datos = obtener_datos_indec_completo()

    if not hojas_datos:
        return None

    # Combinar datos de las 3 hojas
    df_completo = combinar_datos_inflacion(hojas_datos)

    if df_completo.empty:
        print("❌ No se pudieron procesar los datos")
        return None

    # Guardar datos
    df_completo.to_csv("inflacion_argentina_completa.csv", index=False)
    print(f"\n✅ ÉXITO: Datos completos guardados en 'inflacion_argentina_completa.csv'")

    # Mostrar resumen
    mostrar_resumen_completo(df_completo)

    return df_completo

def mostrar_resumen_completo(df):
    """
    Mostrar resumen completo de los datos
    """
    if df is None or df.empty:
        return

    print(f"\n=== RESUMEN COMPLETO DE INFLACIÓN ===")
    print(f"Total registros: {len(df)}")
    print(f"Período: {df['fecha'].min().strftime('%Y-%m')} a {df['fecha'].max().strftime('%Y-%m')}")
    print(f"Columnas disponibles: {list(df.columns)}")

    # Mostrar datos de las columnas disponibles
    columnas_numericas = [col for col in df.columns if col != 'fecha']

    for col in columnas_numericas:
        if col in df.columns and df[col].notna().any():
            print(f"\n{col.upper()}:")
            print(f"  Promedio: {df[col].mean():.2f}")
            print(f"  Máximo: {df[col].max():.2f}")
            print(f"  Mínimo: {df[col].min():.2f}")

    # Últimos 6 meses
    print(f"\n=== ÚLTIMOS 6 MESES ===")
    ultimos = df.tail(6)

    for _, row in ultimos.iterrows():
        fecha = row['fecha'].strftime('%Y-%m')
        linea = f"{fecha}:"

        if 'variacion_mensual' in df.columns and pd.notna(row.get('variacion_mensual')):
            linea += f" {row['variacion_mensual']:+.1f}% mensual"

        if 'variacion_interanual' in df.columns and pd.notna(row.get('variacion_interanual')):
            linea += f" | {row['variacion_interanual']:+.1f}% anual"

        if 'ipc_indice' in df.columns and pd.notna(row.get('ipc_indice')):
            linea += f" | IPC: {row['ipc_indice']:.1f}"

        print(linea)

def obtener_solo_variacion_mensual():
    """
    Función simple para obtener solo variaciones mensuales (más rápida)
    """
    hojas_datos = obtener_datos_indec_completo()

    if 'Variación mensual IPC Nacional' in hojas_datos:
        df_mensual = extraer_nivel_general_de_hoja(
            hojas_datos['Variación mensual IPC Nacional'],
            'Variación mensual'
        )
        if not df_mensual.empty:
            df_mensual.columns = ['fecha', 'variacion_mensual']
            return df_mensual

    return pd.DataFrame()

def obtener_solo_indices():
    """
    Función simple para obtener solo índices IPC
    """
    hojas_datos = obtener_datos_indec_completo()

    if 'Índices IPC Cobertura Nacional' in hojas_datos:
        df_indices = extraer_nivel_general_de_hoja(
            hojas_datos['Índices IPC Cobertura Nacional'],
            'Índices IPC'
        )
        if not df_indices.empty:
            df_indices.columns = ['fecha', 'ipc_indice']
            return df_indices

    return pd.DataFrame()

# Ejecutar
if __name__ == "__main__":
    # Opción completa (todas las hojas)
    df_completo = actualizar_inflacion_completa()

    if df_completo is not None:
        print(f"\n✅ DATOS DISPONIBLES:")
        print("  DataFrame completo: inflacion_argentina_completa.csv")
        print("\n🔧 USO EN CÓDIGO:")
        print("  # Datos completos")
        print("  df = pd.read_csv('inflacion_argentina_completa.csv')")
        print("\n  # O usar funciones específicas:")
        print("  from indec_scraper import obtener_solo_variacion_mensual")
        print("  df_mensual = obtener_solo_variacion_mensual()")

=== OBTENIENDO DATOS COMPLETOS DE INFLACIÓN DEL INDEC ===
Descargando Excel desde: https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_07_25.xls
✓ Archivo descargado: 2283520 bytes
✓ Hoja 0 'Variación mensual IPC Nacional': 215 filas x 103 columnas
✓ Hoja 1 'Var. interanual IPC Nacional': 215 filas x 92 columnas
✓ Hoja 2 'Índices IPC Cobertura Nacional': 215 filas x 104 columnas

=== COMBINANDO DATOS DE LAS 3 HOJAS ===

=== PROCESANDO VARIACIÓN MENSUAL ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 102 registros extraídos

=== PROCESANDO VARIACIÓN INTERANUAL ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 91 registros extraídos

=== PROCESANDO ÍNDICES IPC ===
✓ Fechas en fila 4, Nivel general en fila 8
✓ 103 registros extraídos
✓ DataFrame final: 103 registros
✓ Columnas: ['fecha', 'variacion_mensual', 'variacion_interanual', 'ipc_indice']

✅ ÉXITO: Datos completos guardados en 'inflacion_argentina_completa.csv'

=== RESUMEN COMPLETO DE INFLACIÓN ===
Total registros: 103
Período:

In [56]:
import plotly.graph_objects as go
import pandas as pd

# Asegurarse de que la columna 'fecha' sea de tipo datetime
df_completo['fecha'] = pd.to_datetime(df_completo['fecha'])

# Crear el gráfico de Plotly
fig = go.Figure()

# Agregar la serie de Variación Interanual (línea) - Cambiado a naranja y con zorder
fig.add_trace(go.Scatter(
    x=df_completo['fecha'],
    y=df_completo['variacion_interanual'],
    mode='lines+markers',
    name='Variación Interanual',
    yaxis='y1', # Asociar al eje Y primario
    line=dict(color='orange'), # Color de la línea naranja
    marker=dict(color='orange'), # Color de los marcadores naranja
    zorder=2 # Asegurar que la línea esté adelante (zorder mayor)
))

# Agregar la serie de Variación Mensual (barras) - Cambiado a azul y con zorder
fig.add_trace(go.Bar(
    x=df_completo['fecha'],
    y=df_completo['variacion_mensual'],
    name='Variación Mensual',
    yaxis='y2', # Asociar al eje Y secundario
    marker=dict(color='blue', opacity=0.7), # Color de las barras azules con opacidad
    zorder=1 # Asegurar que las barras estén atrás (zorder menor)
))

# Configurar los ejes y el diseño - Colores de texto en negro y eliminar grilla
fig.update_layout(
    title=dict(
        text='Variación de la Inflación en Argentina (Nivel General) - Interactivo',
        x=0.5,  # Centrar el título (0.5 es el centro horizontal)
        xanchor='center' # Anclar el centro del título al centro horizontal
    ),
    xaxis=dict(
        title='Fecha',
        titlefont=dict(color='black'), # Título del eje X en negro
        tickfont=dict(color='black'), # Etiquetas del eje X en negro
        showgrid=False # Eliminar grilla del eje X
    ),
    yaxis=dict(
        title='Variación Interanual (%)',
        titlefont=dict(color='black'), # Título del eje Y1 en negro
        tickfont=dict(color='black'), # Etiquetas del eje Y1 en negro
        showgrid=False # Eliminar grilla del eje Y1
    ),
    yaxis2=dict(
        title='Variación Mensual (%)',
        titlefont=dict(color='black'), # Título del eje Y2 en negro
        tickfont=dict(color='black'), # Etiquetas del eje Y2 en negro
        overlaying='y', # Superponer al eje Y primario
        side='right',    # Colocar en el lado derecho
        showgrid=False # Eliminar grilla del eje Y2
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    hovermode='x unified', # Mostrar tooltip unificado al pasar el mouse
    plot_bgcolor='white', # Establecer el color de fondo del gráfico a blanco
    font=dict(family='Arial') # Establecer la fuente a Arial
)

# Mostrar el gráfico interactivo
fig.show()